In [1]:
import os

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

## Define static values

In [2]:
folder = "increasing_clients"

# Baseline was estimate using randomly_shuffled_evaluation.py (run 10 x)
baseline = dict({
    "accuracy": 0.2501,
    "kappa": 0.0174,
    "f1-score": 0.1717
})

metrics = ["accuracy", "kappa", "f1-score"]
boxplot_kwargs = {"boxprops": dict(alpha=0.5), "fliersize": 3}
modes = ["central", "federated", "local"]

## Load data

In [3]:
def load_data(model):
    full_df = pd.DataFrame()
    files = os.listdir(folder)
    filtered_files = [f for f in files if f.startswith(model)]
    if filtered_files:
        for filename in filtered_files:
            n_clients = filename.split("_")[1]
            t_df = pd.read_csv(os.path.join(folder, filename))
            t_df["n_clients"] = int(n_clients)
            full_df = pd.concat([full_df, t_df])

    full_df["mode"] = full_df["test_dataset"].str.split('_').str[0]
    return full_df

uni_df = load_data("uni")
conch_df = load_data("conch")

## Figure 5

In [4]:
def create_incr_client_figure(dfs, use_tma20=False):
    fig, axes = plt.subplots(len(dfs), 3, figsize=(10, 5), sharex=True, sharey=True)
    for i, df in enumerate(dfs):
        df = df.dropna()
        for j, metric in enumerate(metrics):
            if use_tma20:
                sns.boxplot(x="n_clients", y=f"{metric}_TMA20", hue="mode", dodge=True, data=df, ax=axes[i][j], **boxplot_kwargs)
            else:
                sns.boxplot(x="n_clients", y=metric, hue="mode", dodge=True, data=df, ax=axes[i][j], **boxplot_kwargs)

            axes[i][j].axhline(baseline[metric], linestyle="--", linewidth=1)
            axes[i][j].set_xlabel("")
            axes[i][j].set_ylabel("")
            axes[i][j].legend().remove()
            axes[i][j].set_ylim(0, 1)

    for i, m in enumerate(metrics):
        axes[0][i].set_title(f"{m.capitalize()}") 

    fig.supylabel("CONCH                               UNI")
    fig.supxlabel("Number of cohorts")

    handles, labels = axes[0][0].get_legend_handles_labels()
    new_handles = [handles[i] for i in range(3)]
    new_labels = [labels[i] for i in range(3)]
    fig.legend(new_handles, new_labels, loc="upper center", title="Training scenario:", ncol=3, bbox_to_anchor=(0.5, 1.05), prop={'style': 'italic'})
    plt.tight_layout(rect=[0, 0, 1, 0.95])

    if use_tma20:
        filename = "increasing_cohorts_TMA20.png"
    else:
        filename = "increasing_cohorts.png"
    plt.savefig(os.path.join(folder, filename), dpi=600, bbox_inches="tight")
    plt.close()

create_incr_client_figure([uni_df, conch_df])

## Supplement

In [5]:
create_incr_client_figure([uni_df, conch_df], use_tma20=True)

## Results and Discussion

In [6]:
def calc_perc_drop_mean(df, mode):
    t_df = df[df["mode"] == mode]
    max_m = ""
    max_p = 0.0
    for m in metrics:
        means = list()
        for n_c in df["n_clients"].unique():
            if n_c == 1:
                pass
            else:
    
                means.append(t_df[t_df["n_clients"] == n_c][m].mean())

        perc = (max(means)-min(means)) / max(means) * 100
        if perc > max_p:
            max_m = m
            max_p = perc
    
    print(f"Max. perc. difference for {max_m}: {max_p:.1f}")

print("central")
calc_perc_drop_mean(uni_df, "central")
calc_perc_drop_mean(conch_df, "central")

print()
print("federated")
calc_perc_drop_mean(uni_df, "federated")
calc_perc_drop_mean(conch_df, "federated")

central
Max. perc. difference for accuracy: 3.9
Max. perc. difference for accuracy: 3.0

federated
Max. perc. difference for kappa: 1.8
Max. perc. difference for kappa: 2.2


In [7]:
print("local")
calc_perc_drop_mean(uni_df, "local")
calc_perc_drop_mean(conch_df, "local")

local
Max. perc. difference for f1-score: 72.7
Max. perc. difference for f1-score: 69.7


In [8]:
def calc_std_incr(df, mode):
    t_df = df[df["mode"] == mode]
    for m in metrics:
        std_3 = t_df[t_df["n_clients"] == 3][m].std()
        std_40 = t_df[t_df["n_clients"] == 40][m].std()

        percentage_increase = ((std_40 - std_3) / std_3) * 100
        print(f"Percentage Increase: {percentage_increase:.2f}%")



print("central")
calc_std_incr(uni_df, "central")
calc_std_incr(conch_df, "central")

print()
print("federated")
calc_std_incr(uni_df, "federated")
calc_std_incr(conch_df, "federated")

central
Percentage Increase: 130.23%
Percentage Increase: 250.35%
Percentage Increase: 158.41%
Percentage Increase: 197.85%
Percentage Increase: 304.24%
Percentage Increase: 245.13%

federated
Percentage Increase: 178.73%
Percentage Increase: 364.55%
Percentage Increase: 209.56%
Percentage Increase: 230.18%
Percentage Increase: 349.64%
Percentage Increase: 304.48%


In [9]:
def calc_std_incr(df, mode):
    t_df = df[df["mode"] == mode]
    for m in metrics:
        m = f"{m}_TMA20"
        std_3 = t_df[t_df["n_clients"] == 3][m].std()
        std_40 = t_df[t_df["n_clients"] == 40][m].std()

        percentage_increase = ((std_40 - std_3) / std_3) * 100
        print(f"Percentage Increase: {percentage_increase:.2f}%")

print("central")
calc_std_incr(uni_df, "central")
calc_std_incr(conch_df, "central")

print()
print("federated")
calc_std_incr(uni_df, "federated")
calc_std_incr(conch_df, "federated")

central
Percentage Increase: -20.61%
Percentage Increase: -12.57%
Percentage Increase: -27.88%
Percentage Increase: 4.77%
Percentage Increase: -28.39%
Percentage Increase: 6.13%

federated
Percentage Increase: -8.76%
Percentage Increase: -17.08%
Percentage Increase: -9.86%
Percentage Increase: -12.06%
Percentage Increase: -5.12%
Percentage Increase: -5.24%
